In [19]:
import sys, os
import pandas as pd
import pathSetting
sys.path.append("../../..")

from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import model as ml_model
from Clust.clust.ML.tool import clean as ml_clean
from Clust.clust.ML.tool import meta as ml_meta

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient

import torch

db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cuda is available.


In [20]:
# Set regression mode i.e., 'regression','forecast' 
mode_selection = "regression"

# Set model methods i.e., 'LSTM_rg', 'GRU_rg', 'CNN_1D_rg', 'LSTM_FCNs_rg', 'FC_rg' 
model_method = 'LSTM_rg'

# get integrated data name
bucket_name = 'integration'

# scaler path
scalerPath = './scaler/'

# 2. Training 

## 2-1. Data selection

In [3]:
get_ms_list = db_client.measurement_list(bucket_name)
get_ms_list

['forecasting_Hs2SwineFarmWithWeatherTime_test_cleanLevel0',
 'forecasting_Hs2SwineFarmWithWeatherTime_test_cleanLevel4',
 'forecasting_Hs2SwineFarmWithWeatherTime_train_cleanLevel0',
 'forecasting_Hs2SwineFarmWithWeatherTime_train_cleanLevel4',
 'forecasting_gunwiStrawberryWeather_test_cleanLevel0',
 'forecasting_gunwiStrawberryWeather_test_cleanLevel4',
 'forecasting_gunwiStrawberryWeather_train_cleanLevel0',
 'forecasting_gunwiStrawberryWeather_train_cleanLevel4',
 'forecasting_strawberryOpen_test_cleanLevel0',
 'forecasting_strawberryOpen_test_cleanLevel4',
 'forecasting_strawberryOpen_train_cleanLevel0',
 'forecasting_strawberryOpen_train_cleanLevel4',
 'regression_energy_testX_cleanLevel0',
 'regression_energy_testX_cleanLevel4',
 'regression_energy_testy_cleanLevel0',
 'regression_energy_testy_cleanLevel4',
 'regression_energy_trainX_cleanLevel0',
 'regression_energy_trainX_cleanLevel4',
 'regression_energy_trainy_cleanLevel0',
 'regression_energy_trainy_cleanLevel4']

In [4]:
collection_list = mongo_client.get_collection_list(bucket_name)
collection_list

['forecasting_strawberryOpen',
 'regression_energy',
 'forecasting_Hs2SwineFarmWithWeatherTime',
 'forecasting_gunwiStrawberryWeather']

In [5]:
# for regression data
# clean_level0 = noclean / clean_level4 = clean
if mode_selection == 'regression':
    # dataX
    data_name_X = 'regression_energy_trainX_cleanLevel0'
    dataX = db_client.get_data(bucket_name, data_name_X)

    # datay
    data_name_y = 'regression_energy_trainy_cleanLevel0'
    datay = db_client.get_data(bucket_name, data_name_y)

    dataset_name = collection_list[1]
    data_meta = mongo_client.get_document_by_json('integration', dataset_name, {'data_name':data_name_X})[0]
    clean_level = data_meta["clean_level"]
    integration_freq_sec = data_meta["integration_param"]["integration_frequency"]

# for forecast data test
elif mode_selection == 'forecast':

    data_name_X = 'forecasting_Hs2SwineFarmWithWeatherTime_train_cleanLevel0'
    dataX = db_client.get_data(bucket_name, data_name_X)
    
    dataset_name = collection_list[0]
    data_meta = mongo_client.get_document_by_json('integration', dataset_name, {'data_name':data_name_X})[0]
    clean_level = data_meta["clean_level"]
    integration_freq_sec = data_meta["integration_param"]["integration_frequency"]

In [6]:
print(type(dataX), dataX.shape)
print(dataX)

<class 'pandas.core.frame.DataFrame'> (13680, 24)
                           Press_mm_hg       RH_1       RH_2       RH_3  \
time                                                                      
2016-01-11 00:00:00+00:00   758.000000  31.666667  31.566667  34.230000   
2016-01-11 00:10:00+00:00   758.083333  31.926667  31.700000  34.290000   
2016-01-11 00:20:00+00:00   758.166667  32.060000  31.956667  34.400000   
2016-01-11 00:30:00+00:00   758.250000  32.260000  32.223333  34.400000   
2016-01-11 00:40:00+00:00   758.333333  32.333333  32.450000  34.400000   
...                                ...        ...        ...        ...   
2016-04-14 23:10:00+00:00   757.833333  32.593333  32.290000  35.000000   
2016-04-14 23:20:00+00:00   757.866667  32.066667  31.926667  34.400000   
2016-04-14 23:30:00+00:00   757.900000  31.760000  31.600000  34.126667   
2016-04-14 23:40:00+00:00   757.933333  31.426667  31.463333  34.090000   
2016-04-14 23:50:00+00:00   757.966667  31.533333 

In [7]:
# print(type(datay), datay.shape)
# print(datay)

## 2-2. Training Data Preparation

In [8]:
featureListX= list(dataX.columns)
featureListX

['Press_mm_hg',
 'RH_1',
 'RH_2',
 'RH_3',
 'RH_4',
 'RH_5',
 'RH_6',
 'RH_7',
 'RH_8',
 'RH_9',
 'RH_out',
 'T1',
 'T2',
 'T3',
 'T4',
 'T5',
 'T6',
 'T7',
 'T8',
 'T9',
 'T_out',
 'Tdewpoint',
 'Visibility',
 'Windspeed']

In [9]:
# for regression
if mode_selection == 'regression':
    # 2 Training Data Preparation
    # 2-1
    featureListX= list(dataX.columns)
    featureListy= list(datay.columns)

    # 2-2
#     cleanTrainDataParam = 'NoClean'#  Classification, Regression과 같이 X, y가 분리된 경우에는 현재 고정해서 사용해야함
    # cleanTrainDataParam = clean_level --> clean_level로 변경
    
    # 2-2-1 cleanTrainDataParam == Clean 일 경우
    NaNProcessingParam ={
        "feature_cycle":'Day',
        "feature_cycle_times":1,
        "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
    }
    # 2-3
    scalerParam='scale'
    scaleMethod='minmax'

    # 2-4
    splitRatio = 0.8

    # 2-5
    scalerRootPath_X = os.path.join(scalerPath, data_name_X, str(clean_level))
    scalerRootPath_y = os.path.join(scalerPath, data_name_y, str(clean_level))
    train_x, val_x, X_scalerFilePath = ml_data.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod)
    train_y, val_y, y_scalerFilePath = ml_data.get_train_val_data(datay, featureListy, scalerRootPath_y, splitRatio, scalerParam, scaleMethod)

# for forecast
elif mode_selection == 'forecast':
    # 2-1
    featureListX = ['Temperature', 'out_temp','sin_hour']
    featureListy = 'Temperature'

    # 2-2
#     cleanTrainDataParam = cleanMode --> clean_level 로 변경

    # 2-2-1 cleanMode == Clean 일 때만 활성화
    NaNProcessingParam ={
        "feature_cycle":'Day',
        "feature_cycle_times":1,
        "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
    }

    # 2-3
    scalerParam='scale'
    scaleMethod ='robust'

    # 2-4
    splitRatio = 0.8

    # 2-5
    scalerRootPath_X = os.path.join(scalerPath, data_name_X, str(clean_level))
    train_x, val_x, X_scalerFilePath = ml_data.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod)
    train_x, val_x = ml_clean.delete_low_quality_train_val_data(train_x, val_x, clean_level, NaNProcessingParam)

['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
Make New scaler File
['value']
Make New scaler File


In [10]:
print(type(train_x), train_x.shape)
print(train_x)

<class 'pandas.core.frame.DataFrame'> (10944, 24)
                           Press_mm_hg      RH_1      RH_2      RH_3  \
time                                                                   
2016-01-11 00:00:00+00:00     0.693237  0.127786  0.324722  0.255336   
2016-01-11 00:10:00+00:00     0.695250  0.134942  0.328622  0.258140   
2016-01-11 00:20:00+00:00     0.697262  0.138611  0.336128  0.263281   
2016-01-11 00:30:00+00:00     0.699275  0.144115  0.343927  0.263281   
2016-01-11 00:40:00+00:00     0.701288  0.146133  0.350556  0.263281   
...                                ...       ...       ...       ...   
2016-03-26 23:10:00+00:00     0.754831  0.201174  0.328037  0.253934   
2016-03-26 23:20:00+00:00     0.753623  0.188148  0.320199  0.253934   
2016-03-26 23:30:00+00:00     0.752415  0.185396  0.330209  0.253934   
2016-03-26 23:40:00+00:00     0.751208  0.185396  0.338448  0.256337   
2016-03-26 23:50:00+00:00     0.750000  0.190166  0.343620  0.259168   

             

In [11]:
print(type(val_x), val_x.shape)
print(val_x)
# print(type(train_y), train_y.shape)
# print(train_y)

<class 'pandas.core.frame.DataFrame'> (2736, 24)
                           Press_mm_hg      RH_1      RH_2      RH_3  \
time                                                                   
2016-03-27 00:00:00+00:00     0.586957  0.267957  0.504094  0.338059   
2016-03-27 00:10:00+00:00     0.583736  0.266306  0.502486  0.334943   
2016-03-27 00:20:00+00:00     0.580515  0.266306  0.500877  0.333385   
2016-03-27 00:30:00+00:00     0.577295  0.267957  0.500877  0.336501   
2016-03-27 00:40:00+00:00     0.574074  0.277039  0.507799  0.339461   
...                                ...       ...       ...       ...   
2016-04-14 23:10:00+00:00     0.689211  0.153289  0.345876  0.291323   
2016-04-14 23:20:00+00:00     0.690016  0.138795  0.335251  0.263281   
2016-04-14 23:30:00+00:00     0.690821  0.130355  0.325697  0.250506   
2016-04-14 23:40:00+00:00     0.691626  0.121182  0.321700  0.248793   
2016-04-14 23:50:00+00:00     0.692432  0.124117  0.319555  0.252220   

              

In [12]:
def transDFtoNP_yk(dfX, dfy, windowNum = 0, dim = None):
    """
    Make NumpyArray by input DataFrame.
    if windowNum = 0 ----> slice X by day
    if windowNum = N ----> slice X by windowNum
    
    Example:
        >>> Retunrn 
        ... X.shape (sampleNum, featureNum, sequenceNum )
        ... y.shape (sampleNum, )

    Args:
        dfX (DataFrame): dfX
        dfy (DataFrame): dfy
        windowNum (Interger): windowNum

    Returns:
        numpy array:  X, y
    
    """
    import datetime as dt
    import numpy as np

    if dim == 2:
        X = dfX.to_numpy()
        y = np.array(dfy.squeeze().tolist())
    else:
        X =[]
        y= []

        if windowNum ==0:
            dateList = dfX.index.map(lambda t: t.date()).unique()
            for startDate in dateList:
                endDate  = dt.datetime.combine(startDate, dt.time(23, 59, 59, 59))
                dfX_partial = dfX[startDate:endDate]
                dfy_partial = dfy[startDate:endDate]
                X_partial = dfX_partial.values
                y_partial = dfy_partial.values[0][0]
                X.append (X_partial)
                y.append (y_partial)
        else:
            import math
            roundNum = math.ceil(len(dfX)/windowNum)
            for i in range(roundNum): #This ensures all rows are captured
                dfX_partial = dfX[i*windowNum:(i+1)*windowNum]
                dfy_partial = dfy[i:(i+1)]
                X_partial = dfX_partial.values
                y_partial = dfy_partial.values[0][0]
                X.append (X_partial)
                y.append (y_partial)

        X = np.array(X)
        y = np.array(y)
    
    return X, y

## 2-3. Training Data Transformation

In [13]:
# for regression
if mode_selection == 'regression':
    transformParameter = {}

    # from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf

    trainX, trainy = transDFtoNP_yk(train_x, train_y, windowNum=144)
    valX, valy = transDFtoNP_yk(val_x, val_y, windowNum=144)

# for forecast
elif mode_selection == 'forecast':
    transformParameter = {
    'future_step': 2,
    'past_step': 24,
    'feature_col': featureListX,
    'target_col': featureListy,
    'clean_level': clean_level
    }
    from Clust.clust.transformation.purpose.machineLearning import LSTMData

    LSTMD = LSTMData()
    trainX, trainy = LSTMD.transform_Xy_arr(train_x, transformParameter, transformParameter['clean_level'])
    valX, valy = LSTMD.transform_Xy_arr(val_x, transformParameter, transformParameter['clean_level'])

In [14]:
print(type(trainX), trainX.shape)
print(trainX)

<class 'numpy.ndarray'> (76, 144, 24)
[[[0.69323671 0.12778644 0.32472217 ... 0.10859729 0.609375   0.53846154]
  [0.6952496  0.13494175 0.32862156 ... 0.11010558 0.609375   0.52564103]
  [0.69726248 0.13861114 0.3361279  ... 0.11161388 0.609375   0.51282051]
  ...
  [0.80072464 0.18264379 0.37248976 ... 0.1040724  0.609375   0.38461538]
  [0.80112721 0.18374461 0.37248976 ... 0.11010558 0.609375   0.38461538]
  [0.80152979 0.18007522 0.36859037 ... 0.11613876 0.609375   0.38461538]]

 [[0.48067633 0.33263003 0.49590563 ... 0.34841629 0.609375   0.30769231]
  [0.48228663 0.33740024 0.52364705 ... 0.34992459 0.609375   0.28205128]
  [0.48389694 0.37949336 0.53450965 ... 0.35143288 0.609375   0.25641026]
  ...
  [0.23792271 0.32311583 0.51687881 ... 0.37330317 0.609375   0.07692308]
  [0.23832528 0.3238235  0.51984514 ... 0.36199095 0.609375   0.07692308]
  [0.23872786 0.32641828 0.52198284 ... 0.35067873 0.609375   0.07692308]]

 [[0.8647343  0.49720209 0.6941899  ... 0.3800905  0.60937

## 2-4 Set Model Parameters & Train Parameters

In [15]:
seq_len, input_size = trainX.shape[1], trainX.shape[2]
print(input_size, seq_len)
# RNN models (RNN, LSTM, GRU) parameters
if model_method == 'LSTM_rg' or model_method == 'GRU_rg':
    modelParameter = {
        'rnn_type': 'lstm',
        'input_size': input_size, 
        'hidden_size': 64,
        'num_layers': 2,
        'output_dim': 1, 
        'dropout': 0.1, 
        'bidirectional': True
    }
# CNN_1D model parameters
elif model_method == 'CNN_1D_rg':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'output_channels': 64,
    'kernel_size': 3,
    'stride': 1,
    'padding': 0, 
    'dropout': 0.1
    }
# LSTM_FCNs model parameters
elif model_method == 'LSTM_FCNs_rg':
    modelParameter = {
    'input_size': input_size,
    'num_layers': 2,
    'lstm_dropout': 0.4,
    'fc_dropout': 0.1
    }
# FC model parameters
elif model_method == 'FC_rg':
    modelParameter = {
    'input_size': input_size,
    'dropout': 0.1,
    'bias': True
    }

trainParameter = {
    'lr': 1e-4,
    'weight_decay': 1e-6, 
    'device': 'cpu', 
    'n_epochs': 10, 
    'batch_size': 16
}


24 144


In [16]:
modelTags =["model_tag_example"]
trainDataType = "timeseries"
modelPurpose = "regression"

# # 2
trainDataInfo = data_meta

# 3. 모델을 저장할 파일 패스를 생성한다.
model_name = None
if model_name is None:
    model_name = dataset_name + '_' + model_method + '_cleanLevel' + str(clean_level)
else:
    pass

print(model_name)

from Clust.clust.transformation.general.dataScaler import encode_hash_style
trainParameter_encode =  encode_hash_style(str(trainParameter))
trainDataPathList = [model_name, data_name_X, trainParameter_encode]
modelFilePath = ml_model.get_model_file_path(trainDataPathList, model_method)

regression_energy_LSTM_rg_cleanLevel0
./Models/LSTM_rg/regression_energy_LSTM_rg_cleanLevel0/regression_energy_trainX_cleanLevel0/d531eec11664669cff1f6a3ad9639012/model.pkl


## 2-5 Training 

In [17]:
from Clust.clust.ML.regression_YK.train import RegressionTrain as RML

rml = RML()
rml.set_param(trainParameter)
rml.set_model(model_method, modelParameter)
rml.set_data(trainX, trainy, valX, valy)
rml.train()
rml.save_best_model(modelFilePath)

Start training model
[1/10] Training loss: 0.2020	 Validation loss: 0.1540
[2/10] Training loss: 0.2025	 Validation loss: 0.1418
[3/10] Training loss: 0.1849	 Validation loss: 0.1300
[4/10] Training loss: 0.1679	 Validation loss: 0.1185
[5/10] Training loss: 0.1576	 Validation loss: 0.1072
[6/10] Training loss: 0.1468	 Validation loss: 0.0963
[7/10] Training loss: 0.1246	 Validation loss: 0.0856
[8/10] Training loss: 0.1282	 Validation loss: 0.0751
[9/10] Training loss: 0.1089	 Validation loss: 0.0653
[10/10] Training loss: 0.0974	 Validation loss: 0.0558

Training complete in 0m 4s


## 2-6 Save MetaData 

In [18]:
# from Clust.clust.transformation.general.dataScaler import encode_hash_style
# trainParameter_encode =  encode_hash_style(str(trainParameter))
if mode_selection == 'regression':
    modelInfoMeta ={
        "trainDataInfo":trainDataInfo,
        "modelName":model_name,
        "featureList":featureListX,
        "target": featureListy,
        "trainDataType":trainDataType,
        "modelPurpose":modelPurpose,
        "model_method":model_method,
        "modelTags":modelTags,
        "cleanLevel":clean_level,
        "NaNProcessingParam":NaNProcessingParam,
        "trainDataName":[data_name_X, data_name_y],
        "trainParameter": trainParameter,
        "modelParameter": modelParameter,
        "transformParameter":transformParameter,
        "NaNProcessingParam":NaNProcessingParam,
        "scalerParam":scalerParam,
        "files":{
            "modelFile":{
                "fileName":"model.pth",
                "filePath":modelFilePath
            },
            "XScalerFile":{
                "fileName":"scaler.pkl",
                "filePath":X_scalerFilePath       
            },
            "yScalerFile":{
                "fileName":"scaler.pkl",
                "filePath":y_scalerFilePath       
            }
        }
    }

elif mode_selection == 'forecast':
    modelInfoMeta ={
        "trainDataInfo":trainDataInfo,
        "modelName":model_name,
        "featureList":featureListX,
        "target": featureListy,
        "trainDataType":trainDataType,
        "modelPurpose":modelPurpose,
        "model_method":model_method,
        "modelTags":modelTags,
        "cleanLevel":clean_level,
        "NaNProcessingParam":NaNProcessingParam,
        "trainDataName":[data_name_X],
        "trainParameter": trainParameter,
        "modelParameter": modelParameter,
        "transformParameter":transformParameter,
        "NaNProcessingParam":NaNProcessingParam,
        "scalerParam":scalerParam,
        "files":{
            "modelFile":{
                "fileName":"model.pth",
                "filePath":modelFilePath
            },
            "XScalerFile":{
                "fileName":"scaler.pkl",
                "filePath":X_scalerFilePath       
            }
        }
    }

modelInfoMeta = ml_meta.save_model_meta_data(mongo_client, modelInfoMeta)


========== Data Save Success ==========
{'trainDataInfo': {'data_name': 'regression_energy_trainX_cleanLevel0', 'ingestion_param': {'start_time': '2016-01-11', 'end_time': '2016-04-15', 'ms_list_info': [['life_indoor_environment', 'humidityTrain_10min'], ['life_indoor_environment', 'temperatureTrain_10min'], ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']]}, 'integration_param': {'param': {}, 'method': 'meta', 'integration_duration': 'common', 'integration_frequency': 600}, 'clean_level': 0, 'process_param': {'refine_param': {'removeDuplication': {'flag': False}, 'staticFrequency': {'flag': False, 'frequency': None}}, 'outlier_param': {'certainErrorToNaN': {'flag': False}, 'unCertainErrorToNaN': {'flag': False}}, 'imputation_param': {'flag': False}}}, 'modelName': 'regression_energy_LSTM_rg_cleanLevel0', 'featureList': ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', '